<a href="https://colab.research.google.com/github/alexanderquispe/CausalAI-Course/blob/group3_lab5_erzo/group3_lab5_r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workgroup 5

Authors: Valerie Dube, Erzo Garay, Juan Marcos Guerrero y Matias Villalba

In [ ]:
install.packages("hdm")
install.packages("xtable")
install.packages("randomForest")
install.packages("glmnet")
install.packages("sandwich")
install.packages("xtable")
install.packages("hdm")
install.packages("glmnet")
install.packages("randomForest")
install.packages("rpart")
install.packages("nnet")
install.packages("gbm")
install.packages("rpart.plot")
install.packages("keras")
install.packages("Rtools")
install.packages("ggplot2")
install.packages("WeightIt")
install.packages("cobalt")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘iterators’, ‘foreach’, ‘shape’, ‘Rcpp’, ‘RcppEigen’, ‘glmnet’, ‘checkmate’, ‘Formula’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘zoo’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installin

In [ ]:
library(hdm)
library(xtable)
library(randomForest)
library(glmnet)
library(sandwich)
library(rpart)
library(nnet)
library(gbm)
library(rpart.plot)
library(keras)
library(xtable)
library(glmnet)
library(randomForest)
library(ggplot2)
library(WeightIt)
library(cobalt)

randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.

Loading required package: Matrix

Loaded glmnet 4.1-8

Loaded gbm 2.1.9

This version of gbm is no longer under development. Consider transitioning to gbm3, https://github.com/gbm-developers/gbm3


Attaching package: ‘ggplot2’


The following object is masked from ‘package:randomForest’:

    margin


 cobalt (Version 4.5.5, Build Date: 2024-04-02)



## Replication and Data analysis

In [ ]:
set.seed(1)
rm(list = ls())

### 1. Descriptives

#### 1.1. Descriptive table (vale)

In [ ]:
# Import data and see first observations
data = read.csv("../../data/processed_esti.csv")
head(data)

In [ ]:
str(data)

In [ ]:
by(data[, !names(data) %in% 'y'], data$w, summary)

#### 1.2. Descriptive graphs (vale)

In [ ]:
#Generating propensity score weights for the ATT
w.out <- WeightIt::weightit(w ~ age + gender_male + ethnicgrp_white + partners1 + imd_decile,
                            data = data,
                            method = "glm",
                            estimand = "ATT")

bal.tab(w.out)

In [ ]:
bal.plot(w.out, var.name = "gender_male")

In [ ]:
bal.plot(w.out, var.name = "gender_male")

In [ ]:
bal.plot(w.out, var.name = "age")
bal.plot(w.out, var.name = "imd_decile")

We can see a higher percentage of participants aged between 23 and 27 in the treatment group. Also, there is a higher perceptange of participants aged between 21 and 22 and 27 and 29 in the control group.
In the case of the IMD decile, an index that measures poverty in the UK, we can see the same proportion of participants in each group.

### 2. Linear Regression analysis

#### 2.1. Regression 1: $y = \beta_0 + \beta_1 T + \epsilon$   (Vale)

In [ ]:
lm(formula = y ~ w, data = data)

#### 2.2. Regression 2: $y = \beta_0 + \beta_1 T + \beta_2 X + \epsilon$  (vale)

In [ ]:
lm(formula = y ~ w + age + gender_female + ethnicgrp_white + ethnicgrp_black + ethnicgrp_mixed_multiple + partners1 + postlaunch + imd_decile, data = data)

#### 2.3.



#### 2.4.

### 3. Non-Linear Methods DML

In [ ]:
rm(list=ls())
DML <- as.data.frame(read.table("../../../data/processed_esti.csv", header=T ,sep=","))
#DML <- as.data.frame(read.table("C:/Users/Erzo/Documents/GitHub/CausalAI-Course/data/processed_esti.csv", header=T ,sep=","))

set.seed(1234)
training <- sample(nrow(DML), nrow(DML)*(3/4), replace=FALSE)
data_train <- DML[training,]
data_test <- DML[-training,]
Y_test <- data_test$y

y = as.matrix(data_train[,1])         # outcome: growth rate
d = as.matrix(data_train[,2])         # treatment: initial wealth
x = as.matrix(data_train[,-c(1,2)]) # controls: country characteristics

### DML function for Regression tree

In [ ]:
DML2.for.PLM.tree <- function(data_train, dreg, yreg, nfold=10) {
  nobs <- nrow(data_train) #number of observations
  foldid <- rep.int(1:nfold,times = ceiling(nobs/nfold))[sample.int(nobs)] #define folds indices
  I <- split(1:nobs, foldid)  #split observation indices into folds
  ytil <- dtil <- rep(NA, nobs)
  cat("fold: ")
  for(b in 1:length(I)){
    datitanow=data_train[-I[[b]],-c(2)]
    datitanoy=data_train[-I[[b]],-c(1)]
    datitanowpredict=data_train[I[[b]],-c(2)]
    datitanoypredict=data_train[I[[b]],-c(1)]

    dfit <- dreg(datitanoy) #take a fold out
    yfit <- yreg(datitanow) # take a foldt out
    dhat <- predict(dfit, datitanoypredict ) #predict the left-out fold
    yhat <- predict(yfit, datitanowpredict ) #predict the left-out fold
    dtil[I[[b]]] <- (d[I[[b]]] - dhat) #record residual for the left-out fold
    ytil[I[[b]]] <- (y[I[[b]]] - yhat) #record residial for the left-out fold
    cat(b," ")
  }
  rfit <- lm(ytil ~ dtil)    #estimate the main parameter by regressing one residual on the other
  coef.est <- coef(rfit)[2]  #extract coefficient
  se <- sqrt(vcovHC(rfit)[2,2]) #record robust standard error
  cat(sprintf("\ncoef (se) = %g (%g)\n", coef.est , se))  #printing output
  return( list(coef.est =coef.est , se=se, dtil=dtil, ytil=ytil) ) #save output and residuals
}

### DML function for Boosting Trees

In [ ]:
DML2.for.PLM.boosttree <- function(data_train, dreg, yreg, nfold=10) {
  nobs <- nrow(data_train) #number of observations
  foldid <- rep.int(1:nfold,times = ceiling(nobs/nfold))[sample.int(nobs)] #define folds indices
  I <- split(1:nobs, foldid)  #split observation indices into folds
  ytil <- dtil <- rep(NA, nobs)
  cat("fold: ")
  for(b in 1:length(I)){
    datitanow=data_train[-I[[b]],-c(2)]
    datitanoy=data_train[-I[[b]],-c(1)]
    datitanowpredict=data_train[I[[b]],-c(2)]
    datitanoypredict=data_train[I[[b]],-c(1)]

    dfit <- dreg(datitanoy) #take a fold out
    best.boostt  <- gbm.perf(dfit, plot.it = FALSE) # cross-validation to determine when to stop
    yfit <- yreg(datitanow) # take a foldt out
    best.boosty  <- gbm.perf(yfit, plot.it = FALSE) # cross-validation to determine when to stop
    dhat <- predict(dfit, datitanoypredict, n.trees=best.boostt)
    yhat <- predict(yfit, datitanowpredict, n.trees=best.boosty) #predict the left-out fold
    dtil[I[[b]]] <- (d[I[[b]]] - dhat) #record residual for the left-out fold
    ytil[I[[b]]] <- (y[I[[b]]] - yhat) #record residial for the left-out fold
    cat(b," ")
  }
  rfit <- lm(ytil ~ dtil)    #estimate the main parameter by regressing one residual on the other
  coef.est <- coef(rfit)[2]  #extract coefficient
  se <- sqrt(vcovHC(rfit)[2,2]) #record robust standard error
  cat(sprintf("\ncoef (se) = %g (%g)\n", coef.est , se))  #printing output
  return( list(coef.est =coef.est , se=se, dtil=dtil, ytil=ytil) ) #save output and residuals
}

### DML function for Lasso and Random Forest

In [ ]:
DML2.for.PLM <- function(x, d, y, dreg, yreg, nfold=10) {
  nobs <- nrow(x) #number of observations
  foldid <- rep.int(1:nfold,times = ceiling(nobs/nfold))[sample.int(nobs)] #define folds indices
  I <- split(1:nobs, foldid)  #split observation indices into folds
  ytil <- dtil <- rep(NA, nobs)
  cat("fold: ")
  for(b in 1:length(I)){
    dfit <- dreg(x[-I[[b]],], d[-I[[b]]]) #take a fold out
    yfit <- yreg(x[-I[[b]],], y[-I[[b]]]) # take a foldt out
    dhat <- predict(dfit, x[I[[b]],], type="response") #predict the left-out fold
    yhat <- predict(yfit, x[I[[b]],], type="response") #predict the left-out fold
    dtil[I[[b]]] <- (d[I[[b]]] - dhat) #record residual for the left-out fold
    ytil[I[[b]]] <- (y[I[[b]]] - yhat) #record residial for the left-out fold
    cat(b," ")
  }
  rfit <- lm(ytil ~ dtil)    #estimate the main parameter by regressing one residual on the other
  coef.est <- coef(rfit)[2]  #extract coefficient
  se <- sqrt(vcovHC(rfit)[2,2]) #record robust standard error
  cat(sprintf("\ncoef (se) = %g (%g)\n", coef.est , se))  #printing output
  return( list(coef.est =coef.est , se=se, dtil=dtil, ytil=ytil) ) #save output and residuals
}